# Creating a feature matrix from a DB Query

In [1]:
import sys
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
from sklearn import preprocessing
import sys
import os
%matplotlib inline 

## Retrieve names of sessions you want from the DB

In [2]:
#load in data base
db = pd.read_csv('/Users/shayneufeld/GitHub/mouse_bandit/session_record.csv',index_col=0)

### Query all 80-20 sessions where performance > 0.7 and block structure was 50

In [5]:
r = db[((db['Left Reward Prob'] == 0.8) |  (db['Right Reward Prob'] == 0.8))].copy()
r = r[r['p(high Port)'] > 0.85].copy()
r = r[r['Block Range Min'] == 50].copy()
r = r[r['Mouse ID'] == 'harry']
session_names = r['Session ID'].values

In [6]:
r

,Session ID,Mouse ID,Date,Phase,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
167,07252016_harry,harry,2016-07-25 00:00:00,2.0,0.2,0.8,50.0,50.0,579.0,8.0,419.0,0.88,2.0,1.0,45.0,40.0
475,07082016_harry,harry,2016-07-08 00:00:00,2.0,0.8,0.2,50.0,50.0,538.0,8.0,401.0,0.88,2.0,1.0,45.0,40.0
496,07132016_harry,harry,2016-07-13 00:00:00,2.0,0.8,0.2,50.0,50.0,653.0,8.0,445.0,0.89,2.0,1.0,45.0,40.0
517,07182016_harry,harry,2016-07-18 00:00:00,2.0,0.8,0.2,50.0,50.0,520.0,7.0,360.0,0.88,2.0,1.0,45.0,40.0
524,07192016_harry,harry,2016-07-19 00:00:00,2.0,0.8,0.2,50.0,50.0,800.0,11.0,574.0,0.87,2.0,1.0,45.0,40.0


In [7]:
r.shape

(5, 16)

## load in csv files (from running exportTrials.m)

In [8]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given']

root_dir = '/Users/shayneufeld/GitHub/mouse_bandit/data/trial_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [9]:
len(trial_df)

21

## convert into 1 feature matrix

In [10]:
for i,df in enumerate(trial_df):
    
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    

In [11]:
master_matrix.shape

(3040, 50)

In [12]:
master_matrix['Higher p port'].mean()

0.88092105263157894

## Save combined feature matrix  

In [13]:
master_matrix.to_csv(os.path.join(root_dir,'harry_8020_high.csv'))